In [106]:
import gym
import numpy as np
import time
from IPython import display

# Thử nghiệm với FrozenLake-v0

In [129]:
env = gym.make('FrozenLake-v0')

In [108]:
env.P[0][3] # Transition model

[(0.3333333333333333, 1, 0.0, False),
 (0.3333333333333333, 0, 0.0, False),
 (0.3333333333333333, 0, 0.0, False)]

In [109]:
env.observation_space.n

16

In [110]:
env.action_space.n

4

In [111]:
def play(env, policy, render=False):
    state = env.reset()
    total_reward = 0
    steps = 0
    done = False
    while not done:
        action = policy[state]
        next_state, reward, done, info = env.step(action)
        total_reward += reward
        steps += 1
        if render:
            env.render()
            time.sleep(0.2)
            if not done:
                display.clear_output(wait=True)
        state = next_state

    return (total_reward, steps)

In [112]:
def play_multiple_times(env, policy, max_episodes):
    success = 0
    list_of_steps = []
    for i in range(max_episodes):
        total_reward, steps = play(env, policy)

        if total_reward > 0:
            success += 1
            list_of_steps.append(steps)

    print(f'Number of successes: {success}/{max_episodes}')
    print(f'Average number of steps: {np.mean(list_of_steps)}')

In [113]:
def value_iteration(env, max_iters=500, gamma=0.9):
    # initialize
    v_values = np.zeros(env.observation_space.n)

    for i in range(max_iters):
        prev_v_values = np.copy(v_values)

        # update the v-value for each state
        for state in range(env.observation_space.n):
            q_values = []
            
            # compute the q-value for each action that we can perform at the state
            for action in range(env.action_space.n):
                q_value = 0
                # loop through each possible outcome
                for prob, next_state, reward, done in env.P[state][action]:
                    q_value += prob * (reward + gamma * prev_v_values[next_state])
                
                q_values.append(q_value)
            
            # select the max q-values
            best_action = np.argmax(q_values)
            v_values[state] = q_values[best_action]
        
        # check convergence
        if np.all(np.isclose(v_values, prev_v_values)):
            print(f'Converged at {i}-th iteration.')
            break
    
    return v_values

In [114]:
def policy_evaluation(env, policy, max_iters=500, gamma=0.9):
    # Initialize the values of all states to be 0
    v_values = np.zeros(env.observation_space.n)

    for i in range(max_iters):
        prev_v_values = np.copy(v_values)

        # Update the value of each state
        for state in range(env.observation_space.n):
            action = policy[state]

            # Compute the q-value of the action
            q_value = 0
            for prob, next_state, reward, done in env.P[state][action]:
                q_value += prob * (reward + gamma * prev_v_values[next_state])

            v_values[state] = q_value # update v-value
        
        # Check convergence
        if np.all(np.isclose(v_values, prev_v_values)):
            #print(f'Converged at {i}-th iteration.')
            break
    
    return v_values

In [115]:
def policy_extraction(env, v_values, gamma=0.9):
    # initialize
    policy = np.zeros(env.observation_space.n, dtype=np.int)

    # loop through each state in the environment
    for state in range(env.observation_space.n):
        q_values = []
        # loop through each action
        for action in range(env.action_space.n):
            q_value = 0
            # loop each possible outcome
            for prob, next_state, reward, done in env.P[state][action]:
                q_value += prob * (reward + gamma * v_values[next_state])
            
            q_values.append(q_value)
        
        # select the best action
        best_action = np.argmax(q_values)
        policy[state] = best_action
    
    return policy

In [116]:
def policy_iteration(env, max_iters=500, gamma=0.9):
    # Init policy with all action is zero 
    policy = np.zeros(env.observation_space.n)

    # Repeat until converged
    for i in range(max_iters):
        # evaluate current policy by policy_evaluation
        v_values = policy_evaluation(env=env, policy=policy)
        # one-step look ahead with policy_extraction
        new_policy = policy_extraction(env, v_values=v_values)

        # if new policy is the same as previous one then it is converged, we should break
        if np.all(policy == new_policy):
            print(f'Converged at {i}-th iteration.')
            break
        else:
            policy = new_policy
    return policy

In [117]:
optimal_v_values = value_iteration(env, max_iters=500, gamma=0.9)

Converged at 79-th iteration.


In [131]:
print('Policy iteration:')
time_start = time.time()
pi_policy = policy_iteration(env)
time_end = time.time()
print(f'Time: {time_end - time_start}')
play_multiple_times(env=env, policy=pi_policy, max_episodes=1000)
print('-------------------------------------------------------------')
print('Value iteration:')
time_start = time.time()
optimal_v_values = value_iteration(env)
optimal_policy = policy_extraction(env, optimal_v_values)
time_end = time.time()
print(f'Time: {time_end - time_start}')
play_multiple_times(env, optimal_policy, 1000)

Policy iteration:
Converged at 5-th iteration.
Time: 0.04954791069030762


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  This is separate from the ipykernel package so we can avoid doing imports until


Number of successes: 758/1000
Average number of steps: 36.193931398416886
-------------------------------------------------------------
Value iteration:
Converged at 79-th iteration.
Time: 0.04405808448791504
Number of successes: 730/1000
Average number of steps: 38.00821917808219


In [119]:
optimal_policy
play_multiple_times(env, optimal_policy, 1000)

Number of successes: 764/1000
Average number of steps: 38.40575916230367


# Thử nghiệm với FrozenLake8x8-v0

In [120]:
env = gym.make('FrozenLake8x8-v0')

In [121]:
env.observation_space.n

64

In [122]:
env.action_space.n

4

In [123]:
print('Policy iteration:')
time_start = time.time()
pi_policy = policy_iteration(env)
time_end = time.time()
print(f'Time: {time_end - time_start}')
play_multiple_times(env=env, policy=pi_policy, max_episodes=1000)
print('-------------------------------------------------------------')
print('Value iteration:')
time_start = time.time()
optimal_v_values = value_iteration(env)
optimal_policy = policy_extraction(env, optimal_v_values)
time_end = time.time()
print(f'Time: {time_end - time_start}')
play_multiple_times(env, optimal_policy, 1000)

Policy iteration:


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  This is separate from the ipykernel package so we can avoid doing imports until


Converged at 9-th iteration.
Time: 0.2327895164489746
Number of successes: 731/1000
Average number of steps: 73.20656634746922
-------------------------------------------------------------
Value iteration:
Converged at 117-th iteration.
Time: 0.27042412757873535
Number of successes: 723/1000
Average number of steps: 74.25034578146611


# Thử nghiệm với Taxi-v3

In [124]:
env = gym.make('Taxi-v3')

In [125]:
env.observation_space.n

500

In [126]:
env.action_space.n

6

In [127]:
print('Policy iteration:')
time_start = time.time()
pi_policy = policy_iteration(env)
time_end = time.time()
print(f'Time: {time_end - time_start}')
play_multiple_times(env=env, policy=pi_policy, max_episodes=1000)
print('-------------------------------------------------------------')
print('Value iteration:')
time_start = time.time()
optimal_v_values = value_iteration(env)
optimal_policy = policy_extraction(env, optimal_v_values)
time_end = time.time()
print(f'Time: {time_end - time_start}')
play_multiple_times(env, optimal_policy, 1000)

Policy iteration:


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  This is separate from the ipykernel package so we can avoid doing imports until


Converged at 16-th iteration.
Time: 2.313659906387329
Number of successes: 1000/1000
Average number of steps: 13.009
-------------------------------------------------------------
Value iteration:
Converged at 116-th iteration.
Time: 1.1200292110443115
Number of successes: 1000/1000
Average number of steps: 13.085


# Nhận xét

-Số bước hội tụ của Value iteration và Policy Iteration lần lượt là 79 và 5 (FronzenLake-v0), 117 và 9 (FrozenLake8x8-v0), 116 và 6 (Taxi-v3)
-Number of successes của Value iteration và Policy Iteration qua các env gần như giống nhau

- Value iteration:
  + Chi phí tính toán đắt hơn Policy iteration
  + yêu cầu nhiều iterations để hội tụ

- Policy iteration:
  + Chi phí tính toán rẻ hơn Value iteration
  + yêu cầu ít iterations hơn để hội tụ
